In [1]:
! pip install datasets transformers sacrebleu -qq

In [ ]:
ds_path='../../data/dataset/lc-quad-wikidata-2021-06-20'

In [2]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm

In [3]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['NNQT_question', 'uid', 'subgraph', 'template_index', 'question', 'sparql_wikidata', 'sparql_dbpedia18', 'template', 'paraphrased_question'],
        num_rows: 19293
    })
    test: Dataset({
        features: ['NNQT_question', 'uid', 'subgraph', 'template_index', 'question', 'sparql_wikidata', 'sparql_dbpedia18', 'template', 'paraphrased_question'],
        num_rows: 4781
    })
})

In [5]:
raw_datasets["train"][0]

{'NNQT_question': 'What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}',
 'paraphrased_question': "What is Delta Air Line's periodical literature mouthpiece?",
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'sparql_dbpedia18': 'select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q188920> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P2813> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?obj . ?obj <http://www.wikidata.org/entity/P31> <http://wikidata.dbpedia.org/resource/Q1002697> } ',
 'sparql_wikidata': ' select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 } ',
 'subgraph': 'simple question right',
 'template': ' <S P ?O ; ?O instanceOf Type>',
 'template_index': 65,
 'uid': 19719}

In [6]:
rep_dict = {"ASK": "ask", "WHERE": "where", "SELECT": "select"}
def replace_all(text):
    for i, j in rep_dict.items(): text = text.replace(i, j)
    return text

In [7]:
# Trim + Replace LowerCase + Remove weirdly long Question

In [8]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = replace_all(d['sparql_wikidata']).strip()
            if len(d['question'])<250:
                df = df.append({col: {'en':d['question'], 'sparql': qry}}, ignore_index=True)
            if len(d['paraphrased_question'])<250:
                df = df.append({col: {'en':d['paraphrased_question'], 'sparql': qry}}, ignore_index=True)
        except: pass
    return df

In [9]:
df_test = prepare(raw_datasets["test"])

100%|██████████| 4781/4781 [00:15<00:00, 314.18it/s]


In [10]:
df_test

,translation
0,{'en': 'Who is the {country} for {head of sta...
1,{'en': 'What country is Mahmoud Abbas the head...
2,{'en': 'What was the population of Somalia in ...
3,"{'en': 'As of 2009, how many people lived in S..."
4,{'en': 'Which female actress is the voice over...
...,...
9542,{'en': 'Bernard Devoto received the Pulizer Pr...
9543,{'en': 'What is antonym of of spore print colo...
9544,{'en': 'What is antonym of of spore print colo...
9545,{'en': 'Tell me mixture whose name has the wor...


In [11]:
df_train = prepare(raw_datasets["train"])

100%|██████████| 19293/19293 [00:55<00:00, 344.93it/s]


In [12]:
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [13]:
ds_train

Dataset({
    features: ['translation'],
    num_rows: 38513
})

In [14]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [15]:
mother_ds.save_to_disk(ds_path)